### Imports

In [2]:
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import applications
from keras.models import Model
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import seaborn as sns

### VGG16 - Frozen, 2 dense layers (4096), classification layer

In [3]:
def vgg_tl():
    img_width, img_height = 224, 224

    train_data_dir = '../data/split_images/train'
    validation_data_dir = '../data/split_images/validation'
    epochs = 40
    batch_size = 16

    if K.image_data_format() == 'channels_first':
        input_shape = (3, img_width, img_height)
    else:
        input_shape = (img_width, img_height, 3)

    # Load VGG16 without the top layers
    vgg_conv_base = applications.VGG16(include_top = False, weights = 'imagenet', input_shape = input_shape)

    # Freeze the VGG16 base layers (make them non-trainable)
    for layer in vgg_conv_base.layers:
        layer.trainable = False

    # Create new model
    model = Sequential()

    # Add the VGG16 base to the new model
    model.add(vgg_conv_base)

    # Add the top layers (Flatten, Dense 64, Dense 4) with a dropout between the dense layers
    model.add(Flatten())
    model.add(Dense(4096))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096))
    model.add(Activation('relu'))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['categorical_accuracy'])

    train_datagen = ImageDataGenerator(
          rescale=1./255,
          rotation_range=20,
          width_shift_range=0.2,
          height_shift_range=0.2,
          horizontal_flip=True,
          fill_mode='nearest')

    validation_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
            train_data_dir,
            target_size=(img_width, img_height),
            batch_size=batch_size,
            class_mode='categorical')

    validation_generator = validation_datagen.flow_from_directory(
            validation_data_dir,
            target_size=(img_width, img_height),
            batch_size=batch_size,
            class_mode='categorical',
            shuffle=False)


    # Train the model
    history = model.fit_generator(
          train_generator,
          steps_per_epoch=train_generator.samples/train_generator.batch_size,
          epochs=epochs,
          validation_data=validation_generator,
          validation_steps=validation_generator.samples/validation_generator.batch_size,
          verbose=1)
    
    return model

model = vgg_tl()
model.save('model.h5')

Found 936 images belonging to 4 classes.
Found 312 images belonging to 4 classes.
Epoch 1/40
59/58 [==============================] - 142s 2s/step - loss: 20.9458 - categorical_accuracy: 0.2831 - val_loss: 2.2803 - val_categorical_accuracy: 0.4295
Epoch 2/40
59/58 [==============================] - 141s 2s/step - loss: 1.5821 - categorical_accuracy: 0.3643 - val_loss: 1.7500 - val_categorical_accuracy: 0.5096
Epoch 3/40
59/58 [==============================] - 141s 2s/step - loss: 1.4235 - categorical_accuracy: 0.4156 - val_loss: 1.2820 - val_categorical_accuracy: 0.4615
Epoch 4/40
59/58 [==============================] - 140s 2s/step - loss: 1.3305 - categorical_accuracy: 0.4466 - val_loss: 1.5764 - val_categorical_accuracy: 0.5897
Epoch 5/40
59/58 [==============================] - 141s 2s/step - loss: 1.2403 - categorical_accuracy: 0.4829 - val_loss: 1.1880 - val_categorical_accuracy: 0.5705
Epoch 6/40
59/58 [==============================] - 141s 2s/step - loss: 1.1953 - categorica